In [1]:
import random
import numpy as np
import math
import copy
from sklearn.svm import SVR
def ABC(X_train,Y_train,X_test,Y_test,T,N,trial_limi,max_flip,s,c,Gamma,Epsilon):
    t=1
    all_bee_trial_lis=[0 for i in range(N)]
    Fitness=[]
    Population=[]
    num_dim=math.ceil(max_flip*N)
    sample_num,featur_num=X_train.shape
    Sample_test_num,featur_num=X_test.shape
    p=s/featur_num
    clf=SVR(kernel='rbf',C=c,gamma=Gamma,epsilon=Epsilon)
    best_active=[]
    best_MSE=float("inf")
    #初始食物源
    for indivi in range(N):
        bee=[ 1 if random.uniform(0,1)<p else 0 for i in range(featur_num)]
        Population.append(bee)
    for each_fit in range(N):
        Active_set=[]
        for fea in range(featur_num):
            if Population[each_fit][fea]==1:
                Active_set.append(fea)
        clf.fit(X_train[:,Active_set],Y_train.ravel())
        Y_predict=clf.predict(X_test[:,Active_set]).reshape(Sample_test_num,1)
        MSE=np.sum((Y_predict-Y_test)**2)/Sample_test_num
        if MSE<best_MSE:
            best_MSE=MSE
            best_active=Active_set
        Fitness.append(MSE)
    
    while t<=T:
        
      
        #Employed Bees
        for each_bee in range(N):
            other_bee=random.randint(0,N-1)
            dim_lis=[random.randint(0,featur_num-1) for i in range(num_dim)]
            cur_bee=copy.deepcopy(Population[each_bee])
            for d in dim_lis:
                if Population[each_bee][d]!=Population[other_bee][d]:
                    cur_bee[d]=Population[other_bee][d]
                
            Active_set=[]
            for fea in range(featur_num):
                if cur_bee[fea]==1:
                    Active_set.append(fea)
            clf.fit(X_train[:,Active_set],Y_train.ravel())
            Y_predict=clf.predict(X_test[:,Active_set]).reshape(Sample_test_num,1)
            MSE=np.sum((Y_predict-Y_test)**2)/Sample_test_num
            if MSE<best_MSE:
                best_MSE=MSE
                best_active=Active_set
            if Fitness[each_bee]>MSE:
                Fitness[each_bee]=MSE
                Population[each_bee]=cur_bee
            else:
                all_bee_trial_lis[each_bee]=all_bee_trial_lis[each_bee]+1
        #calculate P
        #print(best_SIC)
        all_fit=np.sum(np.array(Fitness))
        P_lis=[Fitness[i]/all_fit for i in range(N)]
        #OnLOOKER Bee
        for each_onlook_bee in range(N):
            r=random.uniform(0,1)
            if r>P_lis[each_onlook_bee]:
                food_on_source=random.randint(0,N-1)
                dim_lis=[random.randint(0,featur_num-1) for i in range(num_dim)]
                cur_on_bee=copy.deepcopy(Population[each_onlook_bee])
                for d in dim_lis:
                    if Population[each_onlook_bee][d]!=Population[food_on_source][d]:
                        cur_on_bee[d]=Population[food_on_source][d]
                Active_set=[]
                for fea in range(featur_num):
                    if cur_on_bee[fea]==1:
                        Active_set.append(fea)
                clf.fit(X_train[:,Active_set],Y_train.ravel())
                Y_predict=clf.predict(X_test[:,Active_set]).reshape(Sample_test_num,1)
                MSE=np.sum((Y_predict-Y_test)**2)/Sample_test_num
                if MSE<best_MSE:
                    best_MSE=MSE
                    best_active=Active_set
                if Fitness[each_onlook_bee]>MSE:
                    Fitness[each_onlook_bee]=MSE
                    Population[each_onlook_bee]=cur_on_bee
                else:
                    all_bee_trial_lis[each_onlook_bee]=all_bee_trial_lis[each_onlook_bee]+1
        
        #scoute bee
        for each_scou in range(N):
            if all_bee_trial_lis[each_scou]>=trial_limi:
                scou_bee=[ 1 if random.uniform(0,1)<p else 0 for i in range(featur_num)]
                Active_set=[]
                for fea in range(featur_num):
                    if scou_bee[fea]==1:
                        Active_set.append(fea)
                clf.fit(X_train[:,Active_set],Y_train.ravel())
                Y_predict=clf.predict(X_test[:,Active_set]).reshape(Sample_test_num,1)
                new_MSE=np.sum((Y_predict-Y_test)**2)/Sample_test_num
                if new_MSE<best_MSE:
                    best_MSE=new_MSE
                    best_active=Active_set
        t+=1
    return best_active,best_MSE
            

In [2]:
import pandas as pd
import numpy as np
data=pd.read_csv("HuSheepData.csv",encoding='gb18030')
Weaning_Weight=np.array(data)[5,1:].astype(np.float64).reshape(240,1)
Birth_Weight=np.array(data)[4,1:].astype(np.float64).reshape(240,1)
Six_Weight=np.array(data)[6,1:].astype(np.float64).reshape(240,1)
Featur_Sample=np.array(data)[8:,1:].astype(np.float64).T
X_train=Featur_Sample[:170,:]
X_test=Featur_Sample[170:,:]
Birth_Weight_train=Birth_Weight[:170]
Birth_Weight_test=Birth_Weight[170:]
Six_Weight_train=Six_Weight[:170]
Six_Weight_test=Six_Weight[170:]
Weaning_Weight_train=Weaning_Weight[:170]
Weaning_Weight_test=Weaning_Weight[170:]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,7,9,10,11,13,14,15,16,17,18,20,21,22,23,24,27,28,31,34,74,95,96,97,98,99,100,101,102,103,104,105,106,107,108,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,139,141,143,146,148,150,152,153,155,167,169,170,171,172,173,174,175,176,177,178,179,180,191,192,193,194,205,206,207,210,216,217,218,219,220,221,222,223,224,225,226,227,228) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Birth Weight

In [3]:
Birth_best_active,Birth_best_MSE=ABC(X_train,Birth_Weight_train,X_test,Birth_Weight_test,30,20,50,0.1,2000,0.1,0.01,0.00001)

In [6]:
len(Birth_best_active),Birth_best_MSE,Birth_best_active

(1908,
 0.23909707217506962,
 [31,
  59,
  91,
  118,
  179,
  181,
  182,
  190,
  207,
  269,
  284,
  320,
  339,
  387,
  392,
  426,
  452,
  458,
  525,
  638,
  688,
  731,
  737,
  750,
  800,
  808,
  847,
  855,
  868,
  888,
  900,
  921,
  922,
  966,
  1012,
  1022,
  1038,
  1044,
  1048,
  1071,
  1073,
  1081,
  1159,
  1165,
  1171,
  1189,
  1242,
  1257,
  1286,
  1291,
  1297,
  1324,
  1384,
  1404,
  1446,
  1458,
  1484,
  1578,
  1621,
  1708,
  1742,
  1747,
  1756,
  1766,
  1781,
  1784,
  1798,
  1817,
  1851,
  1885,
  1895,
  1896,
  1916,
  1942,
  1958,
  1987,
  2014,
  2028,
  2073,
  2076,
  2130,
  2142,
  2145,
  2188,
  2245,
  2249,
  2294,
  2354,
  2398,
  2410,
  2414,
  2454,
  2477,
  2515,
  2517,
  2522,
  2542,
  2574,
  2588,
  2613,
  2659,
  2669,
  2673,
  2703,
  2713,
  2873,
  2880,
  2918,
  2927,
  2932,
  2958,
  2973,
  2978,
  2983,
  2998,
  3016,
  3021,
  3079,
  3088,
  3091,
  3121,
  3134,
  3190,
  3251,
  3293,
  3309,


In [9]:
col=np.array(data)[:,0]
gene_name=col[8:]
gene=pd.DataFrame(gene_name[Birth_best_active])
gene.to_csv("best_gene_birth_ABC.csv")

# Six_month

In [4]:
Six_best_active,Six_best_MSE=ABC(X_train,Six_Weight_train,X_test,Six_Weight_test,30,20,50,0.1,2000,1,0.01,0.01)

In [7]:
len(Six_best_active),Six_best_MSE,Six_best_active

(1992,
 5.62535287323116,
 [47,
  50,
  63,
  70,
  91,
  101,
  122,
  129,
  134,
  137,
  148,
  166,
  183,
  187,
  238,
  278,
  290,
  293,
  297,
  332,
  347,
  350,
  399,
  412,
  427,
  487,
  514,
  521,
  583,
  632,
  644,
  645,
  687,
  725,
  733,
  775,
  784,
  794,
  813,
  830,
  846,
  855,
  905,
  945,
  1002,
  1004,
  1026,
  1056,
  1060,
  1089,
  1095,
  1105,
  1109,
  1235,
  1288,
  1292,
  1299,
  1307,
  1312,
  1314,
  1320,
  1335,
  1340,
  1411,
  1432,
  1441,
  1449,
  1515,
  1519,
  1525,
  1544,
  1580,
  1603,
  1605,
  1621,
  1722,
  1754,
  1760,
  1792,
  1866,
  1868,
  1877,
  1891,
  1894,
  1895,
  1913,
  1922,
  1939,
  1948,
  1988,
  2016,
  2024,
  2047,
  2064,
  2108,
  2143,
  2192,
  2202,
  2213,
  2215,
  2280,
  2282,
  2328,
  2342,
  2343,
  2361,
  2368,
  2386,
  2408,
  2436,
  2459,
  2462,
  2485,
  2553,
  2579,
  2590,
  2605,
  2607,
  2619,
  2637,
  2664,
  2678,
  2711,
  2750,
  2780,
  2837,
  2876,
  2896,

In [10]:
col=np.array(data)[:,0]
gene_name=col[8:]
gene=pd.DataFrame(gene_name[Six_best_active])
gene.to_csv("best_gene_Six_ABC.csv")

# Weaning Weight

In [5]:
Weaning_best_active,Weaning_best_MSE=ABC(X_train,Weaning_Weight_train,X_test,Weaning_Weight_test,30,20,50,0.1,2000,0.1,0.01,0.00001)

In [8]:
len(Weaning_best_active),Weaning_best_MSE,Weaning_best_active

(1961,
 15.87973448730502,
 [19,
  84,
  109,
  130,
  132,
  140,
  147,
  153,
  161,
  191,
  202,
  209,
  229,
  317,
  370,
  406,
  436,
  452,
  506,
  535,
  558,
  594,
  643,
  663,
  666,
  710,
  731,
  739,
  761,
  779,
  815,
  831,
  833,
  857,
  862,
  872,
  938,
  950,
  966,
  973,
  977,
  1067,
  1071,
  1093,
  1102,
  1176,
  1192,
  1205,
  1212,
  1226,
  1228,
  1234,
  1242,
  1331,
  1352,
  1412,
  1429,
  1439,
  1441,
  1462,
  1476,
  1510,
  1515,
  1533,
  1579,
  1583,
  1591,
  1633,
  1651,
  1653,
  1671,
  1695,
  1762,
  1766,
  1783,
  1804,
  1807,
  1836,
  1891,
  1942,
  1961,
  1971,
  2007,
  2125,
  2288,
  2301,
  2317,
  2337,
  2451,
  2453,
  2579,
  2582,
  2618,
  2633,
  2643,
  2785,
  2790,
  2794,
  2806,
  2829,
  2831,
  2892,
  2907,
  2979,
  3055,
  3071,
  3076,
  3147,
  3172,
  3182,
  3231,
  3247,
  3289,
  3329,
  3342,
  3379,
  3393,
  3413,
  3417,
  3421,
  3425,
  3435,
  3464,
  3488,
  3503,
  3563,
  3579,


In [12]:
col=np.array(data)[:,0]
gene_name=col[8:]
gene=pd.DataFrame(gene_name[Weaning_best_active])
gene.to_csv("best_gene_Weaning_ABC.csv")